In [1]:
import streamlit as st
import PyPDF2
import io
import openai
from openai import OpenAI
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import os

from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_client = openai.Client(api_key=openai_api_key)

In [ ]:
def draft_response(text):
    # Create a chat completion to draft a response to the document
    completion = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Draft a high-quality, clear, accurate, and helpful reply that answers all points raised in the correspondence, quotes any reference numbers and include the date the initial correspondence was sent. Address the response to the MP who sent the enquiry."},
            {"role": "user", "content": f"Draft a substantive reply for this correspondence:\n\n{text}"}
        ]
    )
    return completion.choices[0].message.content

def refine_response(original_text, style_preferences):
    # OpenAI API call to refine the response based on user stylistic preferences
    completion = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": f"Revise the following text according to the user's stylistic preferences: {style_preferences}."},
            {"role": "user", "content": original_text}
        ]
    )
    return completion.choices[0].message.content


In [7]:
def main():
    extracted_text = "HI THERE BUDDY"
    st.session_state['extracted_text'] = extracted_text

    print("Extracted text: ", extracted_text)

    print("Yessir")

    response = draft_response(st.session_state['extracted_text'])
    st.session_state['drafted_response'] = response
    st.subheader("Drafted Response")
    st.write(response)

    # User input for stylistic preferences
    style_preferences = st.text_input("Input your stylistic preferences (e.g., formal, informal, use of specific terms):")
    
    if st.button("Update Response"):
        if 'drafted_response' in st.session_state and style_preferences:
            updated_response = refine_response(st.session_state['extracted_text'], style_preferences)
            st.subheader("Updated Response")
            st.write(updated_response)


In [8]:
if __name__ == "__main__":
    main()

Extracted text:  HI THERE BUDDY
